# Mood Mate
## Overview
Mood Mate is an AI-powered journaling app. Since starting work at an internship this summer, I have not had time during the day to attend therapy. Many people face the same issue, or simply don't have the option to go to therapy. I built Mood Mate to provide a solution to this problem. While it is not a one-to-one replacement for a therapist, it is capable of providing insights and suggestions based on a users journal entries.
## What It Does
This notebook provides the framework for my classification system. By using a BERT model, we can extract sentiment from text. This sentiment can be fed to another model, Google's Gemini, to make insights even deeper.
The model was setup and deployed here. Since it was bundled with AI Studio, I opted to use ML Flow to serve my frontend.


# Imports
For this project, I used PyTorch, as it came prebundled. I attempted to use TensorFlow, but ran into many issues with Keras compatibility.

In [ ]:
import mlflow
import mlflow.pyfunc
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn.functional as F

# EmotionModelWrapper Class
This clas is a custom wrapper for a BERT-based meotion classification model, built using PyTorch and Hugging Face Transformers.
## Class Definition
We inherit from mlflow.pyfunc.PythonModel which allows seamless integration with MLFlow's model serving and inference APIs.
## load_context
This method is called after the model succesfully loads. It selects our device (CUDA in this case), loads the BERT tokenizer, loads our pretrained model, and sends the model to the appropriate device.
## Predict
This method performs inference using our model. We first extract the text data, then tokenize it, run inference on it, and convert the logits to probabilities. Finally, the probabilities are returned as a NumPy array.

In [9]:
class EmotionModelWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)  # Adjust labels as needed
        self.model.to(self.device)
        self.model.eval()

    def predict(self, context, model_input):
        texts = model_input["text"].tolist()
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probs = F.softmax(logits, dim=-1)
        
        return probs.cpu().numpy()


# MLFlow Model Logging
Since MLFlow is integrated with AI Studio, we can use its logging features to keep track of runs, view outputs of the model, and deploy it.
This code logs a custom PyFunc model to an experiment.
We first setup the experiment. If it already exists, it will log another run. Otherwise, it creates a new one with the given name.
Next, we start an MLFlow run and log it.
We also define our Conda environment to ensure compatibility.

In [10]:
experiment_name = "Bert-Pretrained"
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="emotion_model",
        python_model=EmotionModelWrapper(),
        conda_env={
            'name': 'mlflow-env',
            'channels': ['defaults', 'conda-forge'],
            'dependencies': [
                'python=3.8',
                'pip',
                {
                    'pip': [
                        'torch',
                        'transformers',
                        'pandas',
                        'mlflow',
                    ],
                },
            ],
        }
    )
    print(f"Model logged in run: {run.info.run_id}")


2025/06/09 01:04:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged in run: 1667f57cedb84112affb53b4256a0988


# Load and Use Logged Model
Since we saved our model in the last cell, we can now call and use it again.
We provide some test inputs to make sure it is classifying correctly, and perform predictions on it. This allows us to pre-validate outputs before sending the model to production.

In [11]:
run_id = "1667f57cedb84112affb53b4256a0988"
model_uri = f"runs:/{run_id}/emotion_model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

sample_texts = [
    "I feel incredibly sad and alone",
    "This is a joyful day!",
    "I am afraid of the future"
]

input_df = pd.DataFrame({"text": sample_texts})

predictions = loaded_model.predict(input_df)
print(predictions)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[[0.10719202 0.16217898 0.22271936 0.11458917 0.19264369 0.20067672]
 [0.10431828 0.15692514 0.24225228 0.10864547 0.19797336 0.18988553]
 [0.11122926 0.14448543 0.22541153 0.13491076 0.18686368 0.19709933]]


# MLFlow combined Model Registration, Promotion, and Inference Pipeline
this cell performs a complete workflow with MLFlow. It:
1. Registers a logged model
2. Promotes it to production
3. Loads the production model
4. makes test predictions on input text
## Define Run ID
We first need to define the run ID based on our previously created model.
## Model Promotion
In order to deploy the model through MLFlow, we have to promote it to production. We also catch errors related to duplicate models and revert back to the previous model.
## Model Loading
We load the production model using its URI in order to test on sample text.
## Test Predictions
We provide an array of sample texts, wrap it in a dataframe, and run them through the model. This allows us to spot-check the validity of those predictions before actually sending to production.

In [17]:
import mlflow
import mlflow.pyfunc
import pandas as pd
from mlflow.exceptions import MlflowException

run_id = "1667f57cedb84112affb53b4256a0988"
model_name = "EmotionClassifier"
model_uri = f"runs:/{run_id}/emotion_model"

client = mlflow.tracking.MlflowClient()

try:
    result = mlflow.register_model(model_uri=model_uri, name=model_name)
    client.transition_model_version_stage(
        name=model_name,
        version=result.version,
        stage="Production",
        archive_existing_versions=True
    )
    print(f"Model registered and promoted to Production: v{result.version}")
except MlflowException as e:
    print(f"Model might already be registered or another error occurred: {e}")

prod_model_uri = f"models:/{model_name}/production"
model = mlflow.pyfunc.load_model(prod_model_uri)

sample_texts = [
    "I feel incredibly sad and alone",
    "This is a joyful day!",
    "I am afraid of the future"
]
input_df = pd.DataFrame({"text": sample_texts})

predictions = model.predict(input_df)
print("Predictions:\n", predictions)


Registered model 'EmotionClassifier' already exists. Creating a new version of this model...
Created version '4' of model 'EmotionClassifier'.
/tmp/ipykernel_2512/1092276403.py:14: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model registered and promoted to Production: v4


/opt/conda/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predictions:
 [[0.2941581  0.10023008 0.13040689 0.1924062  0.13009815 0.1527006 ]
 [0.32240707 0.1026908  0.12823935 0.19151546 0.11668826 0.1384591 ]
 [0.3327895  0.09975035 0.1295802  0.14300406 0.11933061 0.1755453 ]]
